In [57]:
import cv2
import os
import numpy as np
import pandas as pd
import spacy
from spacy import displacy
import pytesseract
from pytesseract import Output
import matplotlib.pyplot as plt
pytesseract.pytesseract.tesseract_cmd="/home/aman/anaconda3/envs/tallyInvoiceParser.env/bin/tesseract"
os.environ['TESSDATA_PREFIX'] = "/home/aman/anaconda3/envs/tallyInvoiceParser.env/share/tessdata"
import import_ipynb
from textPreProcessor import *
from imagePreProcessor import *
from datetime import datetime
import math

In [74]:
imagesFolderPath = '/home/aman/Documents/Tally/DocumentAI/Code/Tesseract-Modelv3/Dataset/Imgs/'

In [75]:
def preProcessDataFrame(df, image):
    #drop null values
    df.dropna(inplace=True)
    
    #columns having same values throughout removed
    toDrop = ["level", "page_num"]
    df.drop(columns=toDrop, inplace=True)
    
    #rows having text as a stopword removed
    indexesToDrop = []
    stopwords = ["", " ", "  ", "/", ":", "-", ".", ",", "\n", "\t", "\\", "(", ")", "[", "]", "{", "}", "*", "&", "%", "$", "#", "|"]
    for index in df.index:
        if df["conf"][index]<=40:
            indexesToDrop.append(index)
        elif df["text"][index] in stopwords:
            indexesToDrop.append(index)   
    df.drop(indexesToDrop, inplace=True)
    
    #resetting the index
    df.reset_index(inplace=True)
    df.drop(columns=["index"], inplace=True)
    
    #creating new columns for centroid information
    df["x"] = df["left"] + df["width"] 
    df["y"] = df["top"] + df["height"]
    
    df["PageHeight"] = image.shape[0]
    df["PageWidth"] = image.shape[1]
    
    df["isTop"] = 0
    df["isBottom"] = 0
    df["isRight"] = 0
    df["isLeft"] = 0

In [76]:
def findDate(dateDF):
    dates = []
    format = "%d-%m-%Y"
    for index in dateDF.index:
        text = dateDF["text"][index]
        res = True
        try:
            res = bool(datetime.strptime(text, format))
        except ValueError:
            res = False
        if str(res)=="True":
            dates.append(dateDF["text"][index])
    format = "%m-%d-%Y"
    for index in dateDF.index:
        text = dateDF["text"][index]
        res = True
        try:
            res = bool(datetime.strptime(text, format))
        except ValueError:
            res = False
        if str(res)=="True":
            dates.append(dateDF["text"][index])
    format = "%y-%m-%d"
    for index in dateDF.index:
        text = dateDF["text"][index]
        res = True
        try:
            res = bool(datetime.strptime(text, format))
        except ValueError:
            res = False
        if str(res)=="True":
            dates.append(dateDF["text"][index])
    return dates

In [77]:
def extractDateDataFrame(df):
    dateDF = df.copy()
    for index in dateDF.index:
        text = dateDF["text"][index]
        dateDF["text"][index] = preProcessText(text)
    dates = findDate(dateDF)
    return dateDF, dates
    
        

In [78]:
def returnDist(x1, y1, x2, y2):
    p = [x1, y1]
    q = [x2, y2]
    return math.dist(p, q)

In [128]:
def allocateNeighbours(df, neighbourFields):
    for index in df.index:
        x1 = df["x"][index]
        y1 = df["y"][index]
        block = df["block_num"][index]
        para = df["par_num"][index]
        neighbours = {}
        for index2 in df.index:
            x2 = df["x"][index2]
            y2 = df["y"][index2]
            block2 = df["block_num"][index2]
            para2 = df["par_num"][index2]
            dist = returnDist(x1, y1, x2, y2)
            hr = df["PageHeight"][index2]// df["PageWidth"][index2]
            x = df["PageWidth"][index2]*0.2
            y = df["PageHeight"][index2]*0.2*hr
            radius = math.sqrt((x**2) + (y**2))
            if dist<radius:
                neighbours[df["text"][index2].lower()]=dist
            if abs(y1-y2)<df["PageHeight"][index2]*0.05 and abs(x2-x1)<df["PageWidth"][index2] and abs(block2-block)<=1 and abs(para2==para)<=1:
                neighbours[df["text"][index2].lower()] = math.sqrt(abs(y1-y2)**2 + abs(x2-x1)**2)
        for n in neighbourFields:
            if n in neighbours:
                df[n][index]=1

In [129]:
def addNeighbours(df):
    neighbourFields = ["date", "dated", "invoice", "delivery", "order", "due", "payment", "tax", "bill", "receipt", "issue"]
    for col in neighbourFields:
        df[col]=0
    allocateNeighbours(df, neighbourFields)
    

In [130]:
def findDateDF(dates, dateDF):
    indexes = []
    for date in dates:
        i = dateDF[dateDF["text"]==date].index
        for index in i:
            indexes.append(index)
    return indexes

In [131]:
def dropIndexes(indexes, df):
    indexesToDrop=[]
    for index in df.index:
        if index not in indexes:
            indexesToDrop.append(index)
    df.drop(indexesToDrop, inplace=True)

In [132]:
def create(imagePath):
    image = cv2.imread(imagePath)
    preProcessedImage = preProcessImage(imagePath)
    data = pytesseract.image_to_data(preProcessedImage, output_type=Output.DATAFRAME)
    df = pd.DataFrame(data)
    preProcessDataFrame(df, preProcessedImage)
    dateDF, dates = extractDateDataFrame(df)
    addNeighbours(df)
    indexes = findDateDF(dates, dateDF)
    resultDF = df.copy()
    dropIndexes(indexes, resultDF)
    return resultDF
    
    

In [133]:
def batchCreate(folderPath):
    os.chdir(folderPath)
    images = os.listdir()
    dfList = []
    for imagePath in images:
        #print(imagePath)
        df = create(imagePath)
        df["imageName"] = imagePath
        df["output"] = 0
        dfList.append(df)
    return dfList

In [134]:
dfList = batchCreate(imagesFolderPath)

/tmp/ipykernel_27699/2943746453.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dateDF["text"][index] = preProcessText(text)
/tmp/ipykernel_27699/487488896.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[n][index]=1
/tmp/ipykernel_27699/2943746453.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dateDF["text"][index] = preProcessText(text)
/tmp/ipykernel_27699/2943746453.py:5: SettingWithCopyWarning: 
A value is trying t

/tmp/ipykernel_27699/2943746453.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dateDF["text"][index] = preProcessText(text)
/tmp/ipykernel_27699/487488896.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[n][index]=1
/tmp/ipykernel_27699/2943746453.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dateDF["text"][index] = preProcessText(text)
/tmp/ipykernel_27699/2943746453.py:5: SettingWithCopyWarning: 
A value is trying t

/tmp/ipykernel_27699/2943746453.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dateDF["text"][index] = preProcessText(text)
/tmp/ipykernel_27699/487488896.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[n][index]=1
/tmp/ipykernel_27699/2943746453.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dateDF["text"][index] = preProcessText(text)
/tmp/ipykernel_27699/487488896.py:24: SettingWithCopyWarning: 
A value is trying t

/tmp/ipykernel_27699/2943746453.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dateDF["text"][index] = preProcessText(text)
/tmp/ipykernel_27699/2943746453.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dateDF["text"][index] = preProcessText(text)
/tmp/ipykernel_27699/487488896.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[n][index]=1
/tmp/ipykernel_27699/2943746453.py:5: SettingWithCopyWarning: 
A value is trying t

/tmp/ipykernel_27699/2943746453.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dateDF["text"][index] = preProcessText(text)
/tmp/ipykernel_27699/487488896.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[n][index]=1
/tmp/ipykernel_27699/2943746453.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dateDF["text"][index] = preProcessText(text)
/tmp/ipykernel_27699/487488896.py:24: SettingWithCopyWarning: 
A value is trying t

...

/tmp/ipykernel_27699/2943746453.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dateDF["text"][index] = preProcessText(text)
/tmp/ipykernel_27699/487488896.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[n][index]=1
/tmp/ipykernel_27699/2943746453.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dateDF["text"][index] = preProcessText(text)
/tmp/ipykernel_27699/487488896.py:24: SettingWithCopyWarning: 
A value is trying t

/tmp/ipykernel_27699/2943746453.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dateDF["text"][index] = preProcessText(text)
/tmp/ipykernel_27699/487488896.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[n][index]=1
/tmp/ipykernel_27699/2943746453.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dateDF["text"][index] = preProcessText(text)
/tmp/ipykernel_27699/487488896.py:24: SettingWithCopyWarning: 
A value is trying t

/tmp/ipykernel_27699/2943746453.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dateDF["text"][index] = preProcessText(text)
/tmp/ipykernel_27699/487488896.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[n][index]=1
/tmp/ipykernel_27699/2943746453.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dateDF["text"][index] = preProcessText(text)
/tmp/ipykernel_27699/487488896.py:24: SettingWithCopyWarning: 
A value is trying t

/tmp/ipykernel_27699/2943746453.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dateDF["text"][index] = preProcessText(text)
/tmp/ipykernel_27699/487488896.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[n][index]=1
/tmp/ipykernel_27699/2943746453.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dateDF["text"][index] = preProcessText(text)
/tmp/ipykernel_27699/487488896.py:24: SettingWithCopyWarning: 
A value is trying t

/tmp/ipykernel_27699/2943746453.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dateDF["text"][index] = preProcessText(text)
/tmp/ipykernel_27699/487488896.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[n][index]=1
/tmp/ipykernel_27699/2943746453.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dateDF["text"][index] = preProcessText(text)
/tmp/ipykernel_27699/487488896.py:24: SettingWithCopyWarning: 
A value is trying t

In [135]:
len(dfList)

679

In [136]:
resultDF = pd.concat(dfList)

In [137]:
resultDF.shape

(330, 31)

In [138]:
len(resultDF["imageName"].unique())

243

In [139]:
resultDF

,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,...,delivery,order,due,payment,tax,bill,receipt,issue,imageName,output
108,7,9,2,3,300,1105,155,23,96.191299,02.04.2023,...,0,1,0,0,0,0,0,0,20.jpeg,0
114,7,9,3,4,1379,1146,155,23,95.732468,02.04.2023,...,0,1,0,0,1,0,0,0,20.jpeg,0
68,9,3,1,3,588,1037,84,19,94.234421,9/10/2022,...,0,0,0,1,0,0,0,0,23.jpeg,0
69,9,3,1,4,745,1031,88,32,84.635925,16-10-2022,...,0,0,0,1,0,0,0,0,23.jpeg,0
11,3,1,1,3,398,359,115,21,96.491447,11/09/2017,...,0,0,0,0,0,0,0,0,356.jpg,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,14,1,2,5,1337,1086,115,18,88.408142,27-DEC-22,...,0,1,1,1,0,0,0,0,46.jpeg,0
28,5,1,3,3,262,465,140,28,93.644577,11/06/2018,...,0,0,0,0,1,0,0,0,586.jpg,0
25,7,1,1,1,2824,1809,344,127,69.602173,28/04/18,...,0,0,0,0,1,0,0,0,303.jpg,0
30,5,1,1,1,36,568,138,30,93.391312,29/03/18,...,0,0,0,0,1,0,0,0,096.jpg,0


In [140]:
resultDF.reset_index(inplace=True)

In [141]:
resultDF.drop(columns=["index"], inplace=True)

In [142]:
resultDF.head()

,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,...,delivery,order,due,payment,tax,bill,receipt,issue,imageName,output
0,7,9,2,3,300,1105,155,23,96.191299,02.04.2023,...,0,1,0,0,0,0,0,0,20.jpeg,0
1,7,9,3,4,1379,1146,155,23,95.732468,02.04.2023,...,0,1,0,0,1,0,0,0,20.jpeg,0
2,9,3,1,3,588,1037,84,19,94.234421,9/10/2022,...,0,0,0,1,0,0,0,0,23.jpeg,0
3,9,3,1,4,745,1031,88,32,84.635925,16-10-2022,...,0,0,0,1,0,0,0,0,23.jpeg,0
4,3,1,1,3,398,359,115,21,96.491447,11/09/2017,...,0,0,0,0,0,0,0,0,356.jpg,0


In [143]:
pwd

'/home/aman/Documents/Tally/DocumentAI/Code/Tesseract-Modelv3/Dataset/Imgs'

In [144]:
os.chdir('/home/aman/Documents/Tally/DocumentAI/Code/Tesseract-Modelv3/Dataset/')

In [145]:
pwd

'/home/aman/Documents/Tally/DocumentAI/Code/Tesseract-Modelv3/Dataset'

In [146]:
resultDF.to_csv("tesseract.csv")